<a href="https://colab.research.google.com/github/fcoliveira-utfpr/chuva_probabilidade_sh/blob/main/analises_mensal_chuva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Instalando e importando bibliotecas

In [17]:
#Instalando bibliotecas
!pip install pymannkendall -q

#Importando bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
from datetime import timedelta
import math
from scipy import stats
from scipy.stats import (gamma, norm, ks_2samp, beta, weibull_min, expon, lognorm)
from matplotlib.ticker import FuncFormatter
from scipy.stats import ksone
from sklearn.preprocessing import MinMaxScaler
import pymannkendall as mk
from scipy.stats import linregress

##Importando e abrindo planilha

In [18]:
base_url = "https://raw.githubusercontent.com/fcoliveira-utfpr/chuva_probabilidade_sh/refs/heads/main/"
arquivo = "dataset_chuva_sh.csv"
url = base_url + arquivo
df = pd.read_csv(url)
data = df['Data']
df = df.drop(columns=['Data'])
df = df.replace({',': '.'}, regex=True)
df = df.apply(lambda x: pd.to_numeric(x, errors = 'coerce'), axis=1)
df['Data'] = pd.to_datetime(data, format='%d/%m/%Y')
df

,Dia,Mês,Ano,Chuva (mm),lat,lon,alt,Data
0,1.0,1.0,1976.0,0.0,-24.887224,-54.30623,238.0,1976-01-01
1,2.0,1.0,1976.0,0.0,-24.887224,-54.30623,238.0,1976-01-02
2,3.0,1.0,1976.0,0.0,-24.887224,-54.30623,238.0,1976-01-03
3,4.0,1.0,1976.0,0.0,-24.887224,-54.30623,238.0,1976-01-04
4,5.0,1.0,1976.0,0.0,-24.887224,-54.30623,238.0,1976-01-05
...,...,...,...,...,...,...,...,...
17527,27.0,12.0,2023.0,0.0,-24.887224,-54.30623,238.0,2023-12-27
17528,28.0,12.0,2023.0,0.0,-24.887224,-54.30623,238.0,2023-12-28
17529,29.0,12.0,2023.0,9.8,-24.887224,-54.30623,238.0,2023-12-29
17530,30.0,12.0,2023.0,0.0,-24.887224,-54.30623,238.0,2023-12-30


##Agrupando chuva mensal por ano

In [19]:
# Agrupa os dados por ano e mês e calcule o total acumulado mensal
df_acumulado = df.groupby(['Ano', 'Mês'])['Chuva (mm)'].sum().reset_index()
df_acumulado

,Ano,Mês,Chuva (mm)
0,1976.0,1.0,220.5
1,1976.0,2.0,177.5
2,1976.0,3.0,44.5
3,1976.0,4.0,98.4
4,1976.0,5.0,75.1
...,...,...,...
571,2023.0,8.0,88.3
572,2023.0,9.0,111.0
573,2023.0,10.0,187.9
574,2023.0,11.0,266.2


##Encontrando FDP e FDA

In [20]:
# Analisando o melhor ajuste para cada mês
periodo = list(range(1, int(df_acumulado['Mês'].iloc[-1]) + 1))

res = []

for y in periodo:
    df_1 = df_acumulado.loc[df_acumulado['Mês'] == y]

    chuva_men = df_1['Chuva (mm)'].tolist()
    for i in chuva_men:
      i + 1

    chuva_mensal = chuva_men
    # Cria um intervalo de valores para a precipitação
    x = np.linspace(0, max(chuva_mensal), 1000)

    # Distribuição Normal
    mu, std = stats.norm.fit(chuva_mensal)
    pdf_normal = stats.norm.pdf(x, mu, std)
    D_normal, p_value_normal = stats.kstest(chuva_mensal, 'norm', args=(mu, std))
    cdf_normal = stats.norm.cdf(x, mu, std)

    # Distribuição Exponencial
    loc_exp, scale_exp = stats.expon.fit(chuva_mensal)
    pdf_exp = stats.expon.pdf(x, loc=loc_exp, scale=scale_exp)
    D_exp, p_value_exp = stats.kstest(chuva_mensal, 'expon', args=(loc_exp, scale_exp))
    cdf_exp = stats.expon.cdf(x, loc=loc_exp, scale=scale_exp)

    # Distribuição Gama
    shape_gama, loc_gama, scale_gama = stats.gamma.fit(chuva_mensal)
    pdf_gama = stats.gamma.pdf(x, a=shape_gama, loc=loc_gama, scale=scale_gama)
    D_gama, p_value_gama = stats.kstest(chuva_mensal, 'gamma', args=(shape_gama, loc_gama, scale_gama))
    cdf_gama = stats.gamma.cdf(x, a=shape_gama, loc=loc_gama, scale=scale_gama)

    # Distribuição Log-Normal
    shape_ln, loc_ln, scale_ln = stats.lognorm.fit(chuva_mensal)
    pdf_ln = stats.lognorm.pdf(x, s=shape_ln, loc=loc_ln, scale=scale_ln)
    D_ln, p_value_ln = stats.kstest(chuva_mensal, 'lognorm', args=(shape_ln, loc_ln, scale_ln))
    cdf_ln = stats.lognorm.cdf(x, s=shape_ln, loc=loc_ln, scale=scale_ln)

    n = len(chuva_mensal)
    alpha = 0.05  # Nível de significância
    D_critical = ksone.ppf(1 - alpha / 2, n)

    # Cria um dicionário com os dados
    data = {
        "Distribuição": ["Normal", "Exponencial", "Gama", "Log-Normal"],
        "P-valor": [p_value_normal, p_value_exp, p_value_gama, p_value_ln],
        "Dsup": [D_normal, D_exp, D_gama, D_ln],
        "Dcrítico": [D_critical, D_critical, D_critical, D_critical], # Valor de D calculado (Dsup) tem que ser menor que D crítico.
        "Mês": [y, y, y, y],
        "Coeficientes": [
            f"mu={mu:.1f}, std={std:.1f}" if not np.isnan(mu) and not np.isnan(std) else "-",
            f"loc={loc_exp:.1f}, scale={scale_exp:.1f}" if not np.isnan(loc_exp) and not np.isnan(scale_exp) else "-",
            f"shape={shape_gama:.1f}, loc={loc_gama:.1f}, scale={scale_gama:.1f}" if not np.isnan(shape_gama) and not np.isnan(loc_gama) and not np.isnan(scale_gama) else "-",
            f"shape={shape_ln:.1f}, loc={loc_ln:.1f}, scale={scale_ln:.1f}" if not np.isnan(shape_ln) and not np.isnan(loc_ln) and not np.isnan(scale_ln) else "-",
        ]
        }
    res.append(pd.DataFrame(data))

res
result = pd.concat(res)
# Determina o melhor ajuste com base no nível de significância
result["Melhor Ajuste"] = "Não siguinificativo"  # Começa com "Inadequado" e é atualizado se um ajuste for adequado

result.loc[result["P-valor"] > alpha, "Melhor Ajuste"] = result.loc[result["P-valor"] > alpha, "Distribuição"]

# Exiba o DataFrame ordenado por mês
result = result.sort_values(by='Mês').reset_index()
result = result[['Melhor Ajuste', 'P-valor', 'Dsup', 'Dcrítico', 'Mês','Coeficientes']]

#Ordenando pelo menor Dsup de cada mÊs
dat = []
for y in periodo:
    df_1 = result.loc[result['Mês'] == y]
    df_2 = df_1.sort_values(by='Dsup', ascending=True)
    dat.append(pd.DataFrame(df_2))
dat
df_res = pd.concat(dat)

for i in chuva_mensal:
  i - 1
df_res.head()

/usr/local/lib/python3.10/dist-packages/scipy/stats/_continuous_distns.py:6554: RuntimeWarning: overflow encountered in divide
  return np.sum((1 + np.log(shifted/scale)/shape**2)/shifted)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.10/dist-packages/scipy/stats/_distn_infrastructure.py:1997: RuntimeWarning: overflow encountered in divide
  place(output, cond, self._pdf(*goodargs) / scale)


,Melhor Ajuste,P-valor,Dsup,Dcrítico,Mês,Coeficientes
2,Gama,0.884399,0.081180,0.192208,1,"shape=9.2, loc=-83.7, scale=27.1"
3,Log-Normal,0.882841,0.081361,0.192208,1,"shape=0.2, loc=-188.4, scale=345.6"
0,Normal,0.617907,0.105789,0.192208,1,"mu=166.5, std=82.2"
1,Não siguinificativo,0.000231,0.301322,0.192208,1,"loc=11.0, scale=155.5"
7,Log-Normal,0.982115,0.064042,0.192208,2,"shape=0.5, loc=-47.3, scale=183.2"


In [21]:
# Encontrando a melhor função para cada mês
# Cria uma lista para armazenar os resultados
resultados_por_mes = []

# Itera de 1 a 12, representando cada mês
for mes in range(1, 13):
    # Filtra o DataFrame para o mês atual e reseta o índice
    df_mes = df_res.loc[df_res['Mês'] == mes].reset_index()

    # Verifica se existe pelo menos uma linha para o mês atual
    if not df_mes.empty:
        # Seleciona a primeira linha do mês e adiciona à lista de resultados
        resultados_por_mes.append(df_mes.iloc[0])

# Converte a lista de resultados em um novo DataFrame, se desejado
df_resultados = pd.DataFrame(resultados_por_mes)
df_resultados

,index,Melhor Ajuste,P-valor,Dsup,Dcrítico,Mês,Coeficientes
0,2,Gama,0.884399,0.081180,0.192208,1,"shape=9.2, loc=-83.7, scale=27.1"
0,7,Log-Normal,0.982115,0.064042,0.192208,2,"shape=0.5, loc=-47.3, scale=183.2"
0,8,Log-Normal,0.957546,0.070442,0.192208,3,"shape=0.4, loc=-69.4, scale=178.9"
0,12,Normal,0.886913,0.080885,0.192208,4,"mu=146.7, std=92.6"
0,16,Log-Normal,0.592832,0.107941,0.192208,5,"shape=0.4, loc=-101.0, scale=244.1"
0,20,Gama,0.874623,0.082298,0.192208,6,"shape=2.0, loc=-7.4, scale=67.0"
0,26,Gama,0.984447,0.063166,0.192208,7,"shape=1.7, loc=-5.1, scale=57.6"
0,29,Exponencial,0.818831,0.088082,0.192208,8,"loc=0.0, scale=90.3"
0,32,Log-Normal,0.670381,0.101313,0.192208,9,"shape=0.4, loc=-87.4, scale=206.1"
0,38,Gama,0.799774,0.089900,0.192208,10,"shape=7.8, loc=-67.6, scale=35.3"


##Determiando precipitação provável



In [22]:
##Determinação da precipitação provável de cada mês
periodo = list(range(1, int(df_acumulado['Mês'].iloc[-1]) + 1))
nivel_probabilidade = [0.9, 0.8, 0.75, 0.5, 0.25, 0.20, 0.10]

resultados = []

for probabilidade in nivel_probabilidade:
    precipitacao_provavel = []

    for y in periodo:
      df_1 = df_acumulado.loc[df_acumulado['Mês'] == y]

      chuva_men = df_1['Chuva (mm)'].tolist()
      for i in chuva_men:
        i + 1
      chuva_mensal = chuva_men

      df_2 = df_res.loc[df_res['Mês'] == y].reset_index()
      best_ajuste = df_2['Melhor Ajuste'].loc[0]
      x = np.linspace(0, max(chuva_mensal), 1000)
      if best_ajuste == "Normal":
        mu, std = stats.norm.fit(chuva_mensal)
        precipitacao_provavel_mensal = max(0, stats.norm.ppf(probabilidade, mu, std))
        precipitacao_provavel.append(precipitacao_provavel_mensal)
      elif best_ajuste == "Exponencial":
        loc_exp, scale_exp = stats.expon.fit(chuva_mensal)
        precipitacao_provavel_mensal = max(0, stats.expon.ppf(probabilidade, loc_exp, scale_exp))
        precipitacao_provavel.append(precipitacao_provavel_mensal)
      elif best_ajuste == "Gama":
        shape_gama, loc_gama, scale_gama = stats.gamma.fit(chuva_mensal)
        precipitacao_provavel_mensal = max(0, stats.gamma.ppf(probabilidade, shape_gama, loc_gama, scale_gama))
        precipitacao_provavel.append(precipitacao_provavel_mensal)
      elif best_ajuste == "Log-Normal":
        shape_ln, loc_ln, scale_ln = stats.lognorm.fit(chuva_mensal)
        precipitacao_provavel_mensal = max(0, stats.lognorm.ppf(probabilidade, shape_ln, loc_ln, scale_ln))
        precipitacao_provavel.append(precipitacao_provavel_mensal)
      else:
        mu, std = stats.norm.fit(chuva_mensal)
        precipitacao_provavel_mensal = max(0, stats.norm.ppf(probabilidade, mu, std))
        precipitacao_provavel.append(precipitacao_provavel_mensal)
    resultados.append(precipitacao_provavel)
    # Cria um DataFrame com os resultados
colunas = ["10%", "20%", "25%", "50%", "75%", "80%","90%"]
mes_num = [1, 2 , 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
df_resultados = pd.DataFrame(resultados, index=colunas, columns=mes_num)
df_prov = df_resultados.T#.reset_index()
df_prov.rename(columns={"index": "Mês"}, inplace=True)

def subtract_one_if_greater_than_zero(value):
    return max(value - 1, 0)  # Garante que o valor nunca seja negativo

# Aplicar a função a cada elemento do DataFrame
df_prov = df_prov.applymap(subtract_one_if_greater_than_zero)
df_prov

<ipython-input-22-8cdf34620b79>:53: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_prov = df_prov.applymap(subtract_one_if_greater_than_zero)


,10%,20%,25%,50%,75%,80%,90%
1,275.013719,230.822113,215.012291,156.522299,106.211398,94.953136,67.514762
2,312.762879,237.766628,213.550361,134.943334,79.935140,69.080300,44.701050
3,224.409360,177.947470,162.282589,108.483728,67.123812,58.449314,38.142750
4,264.436597,223.688204,208.207710,145.733333,83.258957,67.778462,27.030070
5,319.913528,247.641372,223.553119,142.066614,80.973422,68.366100,39.178320
6,249.012585,189.399052,169.331244,101.952918,54.466616,45.426394,26.150446
7,193.946872,145.119997,128.807931,74.739070,37.727976,30.864903,16.558638
8,206.980999,144.372479,124.217038,61.608519,24.984883,19.155441,8.516689
9,259.821845,202.478036,183.258799,117.767038,68.066518,57.729551,33.669498
10,339.234657,285.436087,266.280396,195.921304,136.218554,122.991992,91.011111


##Analisando a tendência mensal

In [23]:
#Trata os dados para análises e gráficos
anos = list(set(df_acumulado['Ano'].tolist()))

chuva_jan = df_acumulado.loc[df_acumulado['Mês'] == 1]['Chuva (mm)']
chuva_fev = df_acumulado.loc[df_acumulado['Mês'] == 2]['Chuva (mm)']
chuva_mar = df_acumulado.loc[df_acumulado['Mês'] == 3]['Chuva (mm)']
chuva_abr = df_acumulado.loc[df_acumulado['Mês'] == 4]['Chuva (mm)']
chuva_maio = df_acumulado.loc[df_acumulado['Mês'] == 5]['Chuva (mm)']
chuva_jun = df_acumulado.loc[df_acumulado['Mês'] == 6]['Chuva (mm)']
chuva_jul = df_acumulado.loc[df_acumulado['Mês'] == 7]['Chuva (mm)']
chuva_ago = df_acumulado.loc[df_acumulado['Mês'] == 8]['Chuva (mm)']
chuva_set = df_acumulado.loc[df_acumulado['Mês'] == 9]['Chuva (mm)']
chuva_out = df_acumulado.loc[df_acumulado['Mês'] == 10]['Chuva (mm)']
chuva_nov = df_acumulado.loc[df_acumulado['Mês'] == 11]['Chuva (mm)']
chuva_dez = df_acumulado.loc[df_acumulado['Mês'] == 12]['Chuva (mm)']

In [24]:
# Lista com os dados de chuva para cada mês
chuvas = [chuva_jan, chuva_fev, chuva_mar, chuva_abr, chuva_maio, chuva_jun,
          chuva_jul, chuva_ago, chuva_set, chuva_out, chuva_nov, chuva_dez]
nomes_meses = ['Chuva janeiro', 'Chuva fevereiro', 'Chuva março', 'Chuva abril',
                                            'Chuva maio', 'Chuva junho', 'Chuva julho', 'Chuva agosto',
                                            'Chuva setembro', 'Chuva outubro', 'Chuva novembro', 'Chuva dezembro']

# Lista para armazenar os resultados
resultados = []

# Calcular a regressão e o teste Mann-Kendall para cada mês
for mes, chuva in zip(nomes_meses, chuvas):
    # Regressão linear
    slope, intercept, r_value, p_value, std_err = linregress(anos, chuva)

    # Teste de Mann-Kendall
    mk_result = mk.original_test(chuva)

    # Armazenar os resultados
    resultado = {
        'Mês': mes,
        'Slope': slope,
        'Intercept': intercept,
        'p-value Regressão': p_value,
        'R²': r_value**2,
        'Mann-Kendall S': mk_result.s,
        'Mann-Kendall p-value': mk_result.p,
        'Mann-Kendall Tendência': mk_result.trend
    }
    resultados.append(resultado)

# Cria um DataFrame com os resultados
df_results = pd.DataFrame(resultados)
df_results

,Mês,Slope,Intercept,p-value Regressão,R²,Mann-Kendall S,Mann-Kendall p-value,Mann-Kendall Tendência
0,Chuva janeiro,0.800782,-1434.679454,0.360483,0.018209,102.0,0.369350,no trend
1,Chuva fevereiro,1.776764,-3389.854208,0.135127,0.047879,147.0,0.194389,no trend
2,Chuva março,0.257789,-392.063197,0.749675,0.002235,58.0,0.612423,no trend
3,Chuva abril,-0.228419,603.458029,0.817688,0.001167,-55.0,0.631246,no trend
4,Chuva maio,-0.528539,1222.525955,0.658407,0.004286,-20.0,0.865897,no trend
5,Chuva junho,0.658684,-1192.564307,0.524057,0.008881,10.0,0.936243,no trend
6,Chuva julho,-0.337706,769.395736,0.670293,0.003976,-103.0,0.364610,no trend
7,Chuva agosto,-0.390436,871.002556,0.644173,0.004677,-100.0,0.378817,no trend
8,Chuva setembro,-0.690789,1517.626731,0.478692,0.010968,-72.0,0.528007,no trend
9,Chuva outubro,1.068774,-1928.424879,0.296320,0.023684,113.0,0.319493,no trend
